In [1]:
import pandas as pd
import unidecode
from functools import reduce
import pickle

from collections import defaultdict
from statistics import mean, median

In [2]:
whoscored = pd.read_pickle('Whoscored_Historicals.pkl')
transfermarkt = pd.read_pickle("/Users/georgevarelas/Luther/Players_Historicals_With_Injuries.pkl")
whoscored_urls = pd.read_pickle('/Users/georgevarelas/Luther/whoscored_urls_transfermarkt_names')

In [164]:
transfermarkt = transfermarkt[transfermarkt['Name'].isin(whoscored_urls['Name'].unique())]
whoscored_urls = whoscored_urls.rename(index=str, columns={"Player": "URL"})

In [28]:
def Find_Missing_Vals(l):
    return list(set([x for x in l if l.count(x) > 1]))

In [103]:
#merge whoscored and whoscored_urls, because whoscored_urls has the correct names
whoscored = (pd.merge(whoscored, whoscored_urls, on='URL')).drop_duplicates()
whoscored.drop('Player Name', axis = 1, inplace = True)

In [166]:
Find_Missing_Vals(list(transfermarkt['Name']))

['Paulinho',
 'Emerson',
 'Otavio',
 'Rodrigo',
 'Fernando',
 'Marcelo',
 'Danilo',
 'Lucas Lima',
 'Joao Pedro',
 'Pedro']

In [121]:
to_drop_transfermarkt_indices = [184, 1393, 909, 97, 471, 1378, 858, 1388, 130, 325]
to_drop_whoscored_urls = ['/Players/114197/Show/Paulinho','/Players/122089/Show/Paulinho', '/Players/135563/Show/Paulinho', 
                          '/Players/15528/Show/Emerson', '/Players/328512/Show/Emerson', '/Players/146764/Show/Otavio',\
                          '/Players/31958/Show/Fernando', '/Players/121767/Show/Marcelo', '/Players/27761/Show/Marcelo',\
                          '/Players/358367/Show/Marcelo', '/Players/77883/Show/Marcelo', '/Players/106873/Show/Danilo', \
                          '/Players/18727/Show/Danilo', '/Players/15906/Show/Danilo', '/Players/294056/Show/Danilo', \
                          '/Players/121494/Show/Joao-Pedro', '/Players/243212/Show/Joao-Pedro', '/Players/318475/Show/Pedro'] 

In [167]:
transfermarkt = transfermarkt.reset_index()
transfermarkt['index'] = transfermarkt['index'].apply(lambda row: int(row))

In [150]:
#remove players with same names

In [171]:
transfermarkt = transfermarkt[~transfermarkt['index'].isin(to_drop_transfermarkt_indices)]
whoscored = whoscored[~whoscored['URL'].isin(to_drop_whoscored_urls)]
transfermarkt.drop('index', axis = 1, inplace = True)
transfermarkt.drop('Profile_Link', axis = 1, inplace = True)

In [246]:
#Convert injury dictionary to list so we can manipulate it easier

In [282]:
def transform_injury_history(row):
    injuries = defaultdict(int)
    for i in row:
        if i[4] == '-' or i[4] == '?':
            games_missed = 0
        else:
            games_missed = i[4]
        
        season = int('20'+ i[0][:2])
        injuries[season] += int(games_missed)
    return injuries

transfermarkt['injuries'] = transfermarkt['Injury_History'].apply(transform_injury_history)

In [330]:
#Convert Historical Market Vals dictionary to list

In [343]:
def extract_historical_values(row):
    historical_values = defaultdict(list)
    last_months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun']
    current_months = ['Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    for i in row:
        if i[0][:3] in last_months:
            season = int(i[0][-4:])-1
        elif i[0][:3] in current_months:
            season = int(i[0][-4:])
        
        value = int(i[1])
        age = int(i[2])
        
        historical_values[season].append(value)
    
    for i in historical_values.keys():
        historical_values[i] = int(median(historical_values[i]))
    
    return historical_values      

transfermarkt['Historicals'] = transfermarkt['Historical_Values'].apply(extract_historical_values)

In [346]:
#Get Historical Ages

In [354]:
def extract_historical_ages(row):
    historical_ages = defaultdict(list)
    last_months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun']
    current_months = ['Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    for i in row:
        if i[0][:3] in last_months:
            season = int(i[0][-4:])-1
        elif i[0][:3] in current_months:
            season = int(i[0][-4:])
        
        age = int(i[2])
        
        historical_ages[season].append(age)
    
    for i in historical_ages.keys():
        historical_ages[i] = int(mean(historical_ages[i]))
    
    return historical_ages

transfermarkt['Historical_Ages'] = transfermarkt['Historical_Values'].apply(extract_historical_ages)

In [368]:
simple = transfermarkt[['Name', 'Agent','injuries', 'Historicals', 'Historical_Ages']]
simple = simple.reset_index()

In [437]:
def extract_data(row):
    final = pd.DataFrame(columns=['Season','Market_Value','Name','Age','Games_Missed'])
    
    for i in range(row):
        
        new = pd.DataFrame(simple['Historicals'][i].items(), columns = ['Season', 'Market_Value'])
        new['Name'] = simple['Name'][i]
        
        ages = pd.DataFrame(simple['Historical_Ages'][i].items(), columns = ['Season', 'Age'])
        ages['Name'] = simple['Name'][i]

        injuries = pd.DataFrame(simple['injuries'][i].items(), columns = ['Season', 'Games_Missed'])
        injuries['Name'] = simple['Name'][i]
        
        dfs = [new, ages, injuries]
        player_df = reduce(lambda left,right: pd.merge(left,right, how = 'outer',on=['Season', 'Name']), dfs)
        
        final = pd.concat([final, player_df])
    
    return final

In [452]:
final_transfermarkt = extract_data(simple.shape[0])
final_transfermarkt.reset_index(inplace = True)
final_transfermarkt = final_transfermarkt.drop('index', axis = 1)
final_transfermarkt['Games_Missed'].fillna(0, inplace=True)

In [468]:
final_transfermarkt.to_pickle('Transfermarkt_Final.pkl')

In [469]:
whoscored.to_pickle('WhoScored_Final.pkl')

In [505]:
final_transfermarkt = pd.merge(final_transfermarkt, transfermarkt[['Name', 'Position']], on = 'Name') 

In [511]:
#there are still some duplicate names in whoscored dataframe, need to remove those bitchezb

In [517]:
potential_drops = Find_Missing_Vals(list(whoscored_urls['Name'])) 

In [518]:
already_dropped = ['Paulinho',
 'Emerson',
 'Otavio',
 'Rodrigo',
 'Fernando',
 'Marcelo',
 'Danilo',
 'Lucas Lima',
 'Joao Pedro',
 'Pedro']

In [519]:
to_drop = list(set(potential_drops) - set(already_dropped))

In [614]:
drop_urls_final = ['/Players/279541/Show/Matheus-Pereira', '/Players/36260/Show/Pedrinho', '/Players/322643/Show/Pedrinho',\
                  '/Players/101463/Show/Gabriel', '/Players/114095/Show/Gabriel', '/Players/334087/Show/Gabriel', \
                  '/Players/149772/Show/Gabriel', '/Players/102259/Show/Rogerio', '/Players/322854/Show/Fabinho', \
                  '/Players/117622/Show/Fabinho', '/Players/107182/Show/Pablo', '/Players/136919/Show/Luiz-Gustavo', \
                  '/Players/329985/Show/Jorginho', '/Players/146665/Show/Jorginho', '/Players/41226/Show/Marquinhos', 
                  '/Players/26657/Show/Alvaro-Gonzalez', '/Players/83784/Show/Raul-Garcia', '/Players/43922/Show/Eder', \
                  '/Players/244297/Show/Adama-Traore', '/Players/271857/Show/Adama-Traore', '/Players/323230/Show/Koke', \
                  '/Players/118126/Show/Joao-Mario', '/Players/28059/Show/Everton', '/Players/315279/Show/Everton',\
                  '/Players/119540/Show/Luan', '/Players/78057/Show/Luan', '/Players/114948/Show/Luan', '/Players/342609/Show/Rafinha', \
                   '/Players/27333/Show/Rafinha', '/Players/41262/Show/Felipe', '/Players/9973/Show/Felipe', \
                   '/Players/33840/Show/Felipe', '/Players/89875/Show/Wanderson', '/Players/140977/Show/Wanderson', \
                   '/Players/111361/Show/Gerson', '/Players/341745/Show/Marlon', '/Players/41039/Show/Maicon', \
                   '/Players/29954/Show/Maicon', '/Players/130508/Show/Arthur', '/Players/355666/Show/Arthur', \
                   '/Players/59214/Show/Rafael', '/Players/64096/Show/Rafael', '/Players/101772/Show/Willian', \
                   '/Players/43938/Show/Luis-Alberto', '/Players/344667/Show/Cristian-Ramirez', '/Players/358561/Show/Fernandinho', \
                   '/Players/74428/Show/Fernandinho', '/Players/118149/Show/Fernandinho', '/Players/101472/Show/Dudu', \
                   '/Players/121496/Show/Carlos-Eduardo']

In [615]:
whoscored = whoscored[~whoscored['URL'].isin(drop_urls_final)]
merged = pd.merge(whoscored, final_transfermarkt, on = ['Name', 'Season'])

In [ ]:
merged['Market_Value'] = merged['Market_Value'].apply(lambda row: float(row))
merged['Age'] = merged['Age'].apply(lambda row: float(row))

In [623]:
merged.to_pickle('TRANSFERMARKT_WHOSCORED_MERGE.pkl')